In [12]:
%reload_ext autoreload
%autoreload 2

import os
import acled_conflict_analysis as acled
from acled_conflict_analysis import visuals
from acled_conflict_analysis import analysis
from acled_conflict_analysis import extraction

from bokeh.plotting import show, output_notebook
import bokeh
from bokeh.core.validation.warnings import EMPTY_LAYOUT, MISSING_RENDERERS
from bokeh.models import Panel, Tabs, TabPanel

from datetime import date
from datetime import datetime

import geopandas as gpd

from bokeh.core.validation import silence

# Conflict in Sudan

In [2]:
countries_of_interest = ["Sudan"]
START_DATE = "2012-01-01"
END_DATE = "2024-10-1"

In [5]:
data = extraction.acled_api(
    email_address=os.environ.get("ACLED_EMAIL"),
    access_key=os.environ.get("ACLED_KEY"),
    countries=countries_of_interest,
    start_date=START_DATE,
    end_date=END_DATE
)


c:\WBG\Anaconda3\envs\data-goods\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.acleddata.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [6]:
analysis.data_type_conversion(data)

In [20]:
conflict_monthly = analysis.get_acled_by_group(data, columns=['latitude', 'longitude'], freq='MS')
conflict_yearly = analysis.get_acled_by_group(data, columns=['latitude', 'longitude'], freq='YS')
conflict_yearly_national = analysis.get_acled_by_group(data, columns=['country'], freq='YS')
conflict_sub_event_type = analysis.get_acled_by_group(data, columns = ['event_type', 'sub_event_type'], freq='MS')
conflict_event_type = analysis.get_acled_by_group(data, columns = ['country','event_type'], freq='MS')

In [9]:
sudan_adm1 = gpd.read_file('../../data/boundaries/gadm41_SDN_1.shp')
sudan_adm2 = gpd.read_file('../../data/boundaries/gadm41_SDN_2.shp')
sudan_adm3 = gpd.read_file('../../data/boundaries/gadm41_SDN_3.shp')

In [10]:
events_dict = {datetime(2023, 4, 15):'Battle of Khartoum'
               }

In [21]:
# output_file("bokeh_plot.html")
output_notebook()
silence(MISSING_RENDERERS, True)


tabs = []

for idx, type in enumerate(["nrFatalities", "nrEvents"]):
    tabs.append(
        TabPanel(
            child=visuals.get_line_plot(
                conflict_yearly_national,
                f"National Conflict Trends in {countries_of_interest[0]}",
                f"Source: ACLED. Accessed {datetime.today().date().isoformat()}",
                subtitle="",
                category="country",
                measure=type,
                events_dict=events_dict
            ),
            title=type.capitalize(),
        )
    )

tabs = Tabs(tabs=tabs, sizing_mode="scale_both")
show(tabs, warn_on_missing_glyphs=False)

Loading BokehJS ...

**As was the hypothesis, 2023 had had the most number of conflict events (4928) and conflict-related deaths (15931) in Sudan.** The previous peak was in 2020 with 959 conflict-related deaths and 1021 conflict events. In 2024, there are 9115 conflict-related deaths and 4928 conflict events so far. 

In [31]:
from bokeh.plotting import show, output_notebook
import bokeh
from bokeh.core.validation.warnings import EMPTY_LAYOUT, MISSING_RENDERERS
from bokeh.models import Panel, Tabs

output_notebook()

bokeh.core.validation.silence(EMPTY_LAYOUT, True)
bokeh.core.validation.silence(MISSING_RENDERERS, True)

measure_names = {
    "nrEvents": "Number of Conflict Events",
    "nrFatalities": "Number of Fatalities",
}
measure_colors = {"nrEvents": "#4E79A7", "nrFatalities": "#F28E2B"}

measure = 'nrEvents'

tabs = []

for category_type in list(conflict_event_type["event_type"].unique()):
    tabs.append(
        TabPanel(
            child=visuals.get_bar_chart(
                conflict_event_type,
                f"{measure_names[measure]} from {category_type}",
                f"Source: ACLED. Accessed date {datetime.today().date().isoformat()}",
                subtitle="",
                category="event_type",
                measure=measure,
                color_code=measure_colors[measure],
                category_value=category_type,
                events_dict=events_dict
            ),
            title=category_type.title(),
        )
    )

tabs = Tabs(tabs=tabs, sizing_mode="scale_both")
show(tabs, warn_on_missing_glyphs=False)

Loading BokehJS ...

**There were many protests leading up to the Battle of Khartoum from 2019 till 2023 (321 in total).** Sudan has seen consistent violence against civilians. A total of 8855 people died as a result of violence against civilians since 2012. 

In [27]:
from bokeh.plotting import show, output_notebook
import bokeh
from bokeh.core.validation.warnings import EMPTY_LAYOUT, MISSING_RENDERERS
from bokeh.models import Panel, Tabs, TabPanel

output_notebook()

bokeh.core.validation.silence(EMPTY_LAYOUT, True)
bokeh.core.validation.silence(MISSING_RENDERERS, True)


tabs = []
measure_names = {
    "nrEvents": "Number of Conflict Events",
    "nrFatalities": "Number of Fatalities",
}
measure_colors = {"nrEvents": "#4E79A7", "nrFatalities": "#F28E2B"}
# acled_adm0 = get_acled_by_admin(syria_adm2_crs, acled, columns = ['ADM2_EN', 'ADM1_EN'])
for category_type in list(conflict_event_type["event_type"].unique()):
    tabs.append(
        TabPanel(
            child=visuals.get_bar_chart(
                conflict_event_type,
                f"Conflict Fatalities from {category_type}",
                f"Source: ACLED. Accessed date {datetime.today().date().isoformat()}",
                subtitle="",
                category="event_type",
                measure="nrFatalities",
                color_code=measure_colors["nrFatalities"],
                category_value=category_type,
                events_dict=events_dict
            ),
            title=category_type.title(),
        )
    )

tabs = Tabs(tabs=tabs, sizing_mode="scale_both")
show(tabs, warn_on_missing_glyphs=False)

Loading BokehJS ...

In [36]:
conflict_monthly = analysis.convert_to_gdf(conflict_monthly)

In [37]:
sizeCategoryLabels = {
    5: "Less than 25",
    50: "Less than 50",
    200: "More than 50",
}

def get_size_category(item):
    if item < 25:
        return 5
    elif item <= 50:
        return 50
    elif item > 50:
        return 200
    
conflict_monthly['sizeCategory'] = conflict_monthly['nrFatalities'].apply(get_size_category)

In [39]:
from matplotlib.colors import LinearSegmentedColormap

# Define the color gradient segments
colors = [(0.0, "#FFD2C1"), (0.5, "#E36359"), (1.0, "#A62C2B")]

# Create the colormap using LinearSegmentedColormap
cmap = LinearSegmentedColormap.from_list("custom_colormap", colors)

In [55]:
# import matplotlib.pyplot as plt
# import contextily as ctx
# import pandas as pd

# fig, ax = plt.subplots(1, 1, sharex=True, figsize=(10, 8))

# # world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
# # country = world[world['name'] == 'Lebanon']

# date_range = pd.date_range(start="2023-04-01", end="2024-10-01", freq='MS')

# for month in date_range:

#     month_name = month.strftime('%B %Y')

#     conflict_monthly[conflict_monthly['event_date']==month].plot(
#         column="nrFatalities",cmap=cmap, ax=ax, markersize="sizeCategory", marker="o"
#     )
#     sudan_adm1.boundary.plot(ax=ax, color="lightgrey", alpha=0.5, zorder=1)
#     ax.set_title(f"Fatalities in {month_name}")

#     sizes = conflict_monthly[conflict_monthly['event_date']==month].sort_values(by="sizeCategory")["sizeCategory"].unique()

#     #country.plot(ax=ax, color='none', edgecolor='black')
#     ctx.add_basemap(ax, crs=conflict_monthly.crs.to_string(), source=ctx.providers.CartoDB.Positron)


#     handles = []
#     for size in sizes:
#         (handle,) = ax.plot(
#             [],
#             [],
#             marker="o",
#             markersize=size / 10,
#             linestyle="",
#             label=sizeCategoryLabels[size],
#             markerfacecolor="white",
#             markeredgecolor="black",
#         )
#         handles.append(handle)
#     ax.legend(
#         handles=handles,
#         title="Number of fatalities",
#         loc="lower right",
#         frameon=False,
#         bbox_to_anchor=(1.4, 0.2),
#     )

#     plt.figtext(0.28, 0.09, f'Source: ACLED. Accessed {datetime.today().date().isoformat()}', ha="left", fontsize=9, color="black")


#     ax.spines["top"].set_visible(False)
#     ax.spines["bottom"].set_visible(False)
#     ax.spines["left"].set_visible(False)
#     ax.spines["right"].set_visible(False)

#     ax.set_xticks([])
#     ax.set_yticks([])

#     plt.savefig(f'../../docs/images/fatalities_{str(month.date())}.png', dpi=800, bbox_inches='tight');


In [71]:
import os
from PIL import Image
import ipywidgets as widgets
from IPython.display import display, Image as IPImage, clear_output
import pandas as pd

# Directory where images are stored
output_dir = '../../docs/images/conflict/'  # Adjust this path if needed

# Load the image filenames (assuming 'fatalities_2023-04-01' format)
image_files = sorted([f for f in os.listdir(output_dir) if f.startswith('fatalities') and f.endswith('.png')])

# Check if image_files is populated correctly
if not image_files:
    print("No images found in the directory. Please check the output_dir path and ensure images are saved.")

# Adjust the parsing logic based on the filename structure (fatalities_YYYY-MM-DD.png)
try:
    # Extract the year and month from the filename
    months = [pd.to_datetime(f.split('_')[1].split('.')[0]) for f in image_files]
    month_names = [date.strftime('%B %Y') for date in months]
except IndexError as e:
    print(f"Error in extracting dates from filenames: {e}")
    month_names = []  # Fallback if there's an error

# Create an output widget for the image display
image_output = widgets.Output()

# Function to update and display the image
def update_frame(frame):
    try:
        # Clear previous image output, but not the controls
        with image_output:
            clear_output(wait=True)
            
            # Load the image for the current frame
            img_path = os.path.join(output_dir, image_files[frame])
            
            # Display the image using IPython's Image class
            display(IPImage(filename=img_path))

            # Set the title or display relevant information (can print in the output)
        
    except Exception as e:
        print(f"Error displaying image {frame}: {e}")

# Create widgets for interactive control
play_button = widgets.Play(
    interval=1000,  # Speed of the animation in milliseconds (1 second = 1000ms)
    value=0,
    min=0,
    max=len(image_files)-1,
    step=1,
    description="Press play",
    disabled=False
)

# Create a SelectionSlider for the months
if month_names:
    slider = widgets.SelectionSlider(
        options=month_names,
        value=month_names[0],
        description="Month",
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True
    )
    
    # Link the play button to the slider
    widgets.jslink((play_button, 'value'), (slider, 'index'))

    # Update the frame when the slider changes
    def on_value_change(change):
        update_frame(month_names.index(change['new']))

    slider.observe(on_value_change, names='value')

    # Display the play button and slider
    display(widgets.HBox([play_button, slider]))

    # Display the output widget for the images
    display(image_output)

    # Display the first image when the notebook loads
    update_frame(0)
else:
    print("Month names could not be generated due to a filename parsing issue.")


Output()